In [2]:
#install upgraded google-cloud-storage
#pip install --upgrade google-cloud-storage

Requirement already up-to-date: google-cloud-storage in /opt/conda/lib/python3.7/site-packages (1.32.0)
Note: you may need to restart the kernel to use updated packages.


# Access Google Cloud Storage (GCS) data

In [1]:
#Access data files for H23 and parental test samples (first batch)

from google.cloud import storage
#google apis documentation: https://googleapis.dev/python/storage/latest/index.html
bucket_name = 'sbi-sheffels-e'
prefix = 'data/pE2.N54.fastq.00/'

storage_client = storage.Client() 
bucket = storage_client.get_bucket(bucket_name)
#Get list of files in data/pE2.N54.fastq.00 folder of bucket
files = list(storage_client.list_blobs(bucket, prefix=prefix)) #blob = GCS name for file/object
#for file in files:
 #   print(file) #print "blob" names



# Get total size of data files

Needed for long term storage solutions

In [3]:
def get_file_size(bucket_name, blob):
    """Gets Google Cloud storage file size from list_blobs. Adapted from Google Cloud tutorial.
    https://cloud.google.com/storage/docs/viewing-editing-metadata#code-samples"""
    # bucket_name = 'your-bucket-name'
    # blob_name = 'your-object-name'

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    #blob = bucket.get_blob(blob_name) #for using with a string file name, not necessary if already have list of blobs
    file_size = blob.size #size in bytes
    return file_size

Google API python storage: https://googleapis.dev/python/storage/latest/blobs.html
attribute names/functions available for storage objects, etc

In [4]:
#Get total size of data in GB
total_size = 0
for file in files:
    file_size = get_file_size(bucket_name, file)
    total_size += file_size
    #print("File size: " , file_size, "Total size: ", total_size)
print("Final size:", total_size/(10**9), "GB.")

Final size: 341.145714441 GB.


# Examine individual file

In [2]:
#Direct to file in bucket
filepath = 'data/pE2.N54.fastq.00/E2-78710_S1_L004_R1_001.fastq.gz'
filename = 'E2-78710_S1_L004_R1_001.fastq.gz'
blob = storage.blob.Blob(filepath,bucket)
#download blob to AI Platform Notebook instance
with open(filename, "wb") as file_obj:
    blob.download_to_file(file_obj)


In [16]:
#Decompress fastq.gz to examine
#read compressed file and write to txt file in chunks to avoid "clogging" memory
import gzip
import shutil
with gzip.open(filename, 'rb') as f_in:
    with open('file.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [18]:
#print first line of fastq
with open('file.txt', 'r') as f:
    print(f.readline())

@A00735:209:HFYKWDSXY:4:1101:2040:1016 1:N:0:AGTTCAGG+CCAACAGA



In [24]:
#print first 1000 characters of fastq
#(using alternative file open/close code structure from above)
f = open('file.txt', 'r')
print(f.read(1000))
f.close()

@A00735:209:HFYKWDSXY:4:1101:2040:1016 1:N:0:AGTTCAGG+CCAACAGA
CNTTATTCCTTTGGCCCAGACAGCTTTGTTGAGCCTGGTGTCAATGCGCACATCTGGAGTTCCCATCTCCTTCATGGCAAATTTCCGAATCTCTTTGAGTG
+
F#FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@A00735:209:HFYKWDSXY:4:1101:4878:1016 1:N:0:AGTTCAGG+CCAACAGA
CNCTCTCTTCCTCTTGTGCTCTTGCTGGGGCTGGTGGTCCAGGGGTCTTACTCCTTGGAGGCCATGTGGGCCATGAGGTCCACCACCCTGTTGCTGTAGCC
+
F#FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFF
@A00735:209:HFYKWDSXY:4:1101:5529:1016 1:N:0:AGTTCAGG+CCAACAGA
TNTTTTTTTTAGGATTTTTAGTATTTTAATATAATACAGTTCCAATCAGTGTGTCTGGTTCATTTCAGTTCCTTCACTGCCAAACCGGGGGGCAGGGACTG
+
F#:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFF:FFFFFFFFFFF:F,FFFFFFFFFFFFFFFFFFFFF
@A00735:209:HFYKWDSXY:4:1101:10248:1016 1:N:0:AGTTCAGG+CCAACAGA
ANTGAAGCCAGTTGATTAGGGTGCTTAGCTGTTAACTAAGTGTTTGTGGGTTTAAGTCCCATTGGTCTAGTGAGGGCTTAGCTTAATTAAAGTGGCTGATT
+
F#FFFFFFFFFFFFFFFFFFFFFFF